# Comparing Temperatures and Luminosities Between Analyses

This notebook measures global temperatures and luminosities for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of galaxy cluster masses, temperatures and luminosities play an important role in the construction of X-ray mass-observable scaling relations - thus they must also be shown to be consistent with previous work. 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs3p_colour, xcs_cosmo, direct_comparison_plot, find_lims, fit_comp

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 60
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sources import GalaxyCluster
from xga.sources import BaseSource
from xga.xspec import single_temp_apec

## Reading in Sample Files and Declaring XGA ClusterSamples

$\color{red}{\text{NEED TO MENTION/PUBLISH THE OBSIDS THAT WE EXCLUDE FROM USE DUE TO DATA PROBLEMS LIKE FLARING}}$

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

We impose an additional cleaning step on each sample, where we make sure that (for each XMM observation initially associated with a source) at least 70% of a cluster's $R_{500}$ falls on that observation - if this requirement is not met then the observation is excluded. These requirements are set with the `clean_obs=True`, `clean_obs_reg='r500'`, and `clean_obs_threshold=0.7` arguments when a ClusterSample instance is declared.

### SDSSRM-XCS Volume Limited

This is the recent SDSSRM-XCS sample. The temperatures and luminosities are measured by the XCS luminosity-temperature pipeline, and with this we demonstrate that XGA temperatures and luminosities are consistent with existing XCS results.

In order to achieve maximum consistency, we use the XAPA coordinates as the central position for spectrum generation (turning off the XGA peak finder with `use_peak=False`). We have also made sure to use the same cosmology.

$\color{red}{\text{UNBLACKLIST 0763910701 ONCE PAUL HAS RE-REDUCED IT - THAT WILL RE-INCLUDE SDSSXCS-5977}}$

In [2]:
xcs3p = pd.read_csv("../../sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")

In [3]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
z = xcs3p['z'].values
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values
# In arcminutes, ClusterSample declaration will convert to kpc using the provided cosmology
r500 = Quantity(xcs3p['r500'].values, 'arcmin')
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values
r_err = xcs3p['richness_err'].values

# Declaring the actual ClusterSample instance for the XCS sample
xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                         load_fits=True, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                         clean_obs_threshold=0.7)

Setting up Galaxy Clusters:   7%|██▉                                         | 10/150 [00:06<01:29,  1.57it/s]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-2346 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   8%|███▌                                        | 12/150 [00:10<02:58,  1.30s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-11331 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:   9%|███▊                                        | 13/150 [00:11<02:45,  1.21s/it]/mnt/

Setting up Galaxy Clusters:  22%|█████████▋                                  | 33/150 [01:15<09:08,  4.69s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-120 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  25%|██████████▊                                 | 37/150 [01:24<04:14,  2.26s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-103 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  28%|████████████▎                               | 42/150 [01:28<01:39,  1.09it/s]/mnt/pac

Setting up Galaxy Clusters:  51%|██████████████████████▌                     | 77/150 [03:18<08:14,  6.78s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-19467 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  53%|███████████████████████▏                    | 79/150 [04:46<25:44, 21.75s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-1174 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  53%|███████████████████████▍                    | 80/150 [04:47<18:05, 15.51s/it]/mnt/

Setting up Galaxy Clusters:  87%|█████████████████████████████████████▌     | 131/150 [06:22<00:33,  1.75s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-487 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  89%|██████████████████████████████████████▍    | 134/150 [06:24<00:15,  1.01it/s]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/extended.py:221: UserWarning: A point source has been detected in 0404190201 and is very close to the user supplied coordinates of SDSSXCS-10223. It will not be excluded from analysis due to the possibility of a mis-identified cool core
  warnings.warn("A point source has been detected in {o} and is very close to the user supplied "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.

## Running $T_{\rm{X}}$ and $L_{\rm{X}}$ Measurements

The XGA XSPEC functions that we use here all automatically call the XGA SAS interface, so the necessary spectra are generated before the fits begin. As the different samples measure properties within different spatial regions, all the function calls differ slightly.

The results of the fits are stored within the indivual source objects that make up each sample.

###  SDSSRM-XCS

For our comparisons here we wish to measure the temperature ($T_{\rm{X}}$) and luminosity ($L_\rm{X}$; both in the 0.5-2.0 keV and bolometric/0.01-100.0 keV energy bands) within $R_{\rm{500}}$. We fit a `constant*tbabs*apec` model; $\color{red}{\text{with the choices for absorption (`tbabs`) and plasma emission (`tbabs`) consistent with the XCS analysis, though the addition of a multiplicative constant to manage differences in sensitivity is different from the original analysis.}}$

In [ ]:
single_temp_apec(xcs_srcs, xcs_srcs.r500)

Generating products of type(s) spectrum:  20%|█████▋                      | 135/664 [38:01<2:36:25, 17.74s/it]

## Retrieving $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements from the samples

We must extract and judge the quality of the temperature and luminosity measurements that we have made for each of the samples, then later on we will be able to directly compare them. At the same time we make sure that the results from literature are formatted in such a way that we can easily compare them.

### SDSSRM-XCS

In [ ]:
sdss_tx_all = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=False).value
sdss_tx = xcs_srcs.Tx(xcs_srcs.r500, quality_checks=True)

sdss_lxbol_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                             hi_en=Quantity(100.0, 'keV'))
sdss_lxbol = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                         hi_en=Quantity(100.0, 'keV'))

sdss_lx_all = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.5, 'keV'), 
                             hi_en=Quantity(2.0, 'keV'))
sdss_lx = xcs_srcs.Lx(xcs_srcs.r500, quality_checks=True, lo_en=Quantity(0.5, 'keV'), 
                         hi_en=Quantity(2.0, 'keV'))

In [ ]:
# xcs3p_tx = Quantity(xcs3p[['Tx', 'Tx-', 'Tx+']].values, 'keV')
# xcs3p_lx52 = Quantity(xcs3p[['Lx52', 'Lx52-', 'Lx52+']].values*1e+44, 'erg/s')
# xcs3p_lxbol = Quantity(xcs3p[['Lx', 'Lx-', 'Lx+']].values*1e+44, 'erg/s')

xcs3p_tx = []
xcs3p_lx52 = []
xcs3p_lxbol = []

for src in xcs_srcs:
    n = src.name
    rel_row = xcs3p[xcs3p['name'] == n].iloc[0]
    xcs3p_tx.append(Quantity(rel_row[['Tx', 'Tx-', 'Tx+']].values, 'keV'))
    xcs3p_lx52.append(Quantity(rel_row[['Lx52', 'Lx52-', 'Lx52+']].values*1e+44, 'erg/s'))
    xcs3p_lxbol.append(Quantity(rel_row[['Lx', 'Lx-', 'Lx+']].values*1e+44, 'erg/s'))  

xcs3p_tx = Quantity(xcs3p_tx)
xcs3p_lx52 = Quantity(xcs3p_lx52)
xcs3p_lxbol = Quantity(xcs3p_lxbol)

## Direct comparisons between original and XGA measurements

This is the the point of this notebook, making direct comparisons of like for like (or as near as we can achieve) measurements from literature to measurements made using XGA. In each case we plot simple one-to-one comparisons, with a one-to-one dashed line in red to give a reference.

### SDSSRM-XCS

We directly compare temperatures and luminosities (soft band and bolimetric) between the XCS3P pipeline and XGA.

In [ ]:
print('{s} out of {t} XGA Tx measurements were successful'.format(s=np.isfinite(sdss_tx[:, 0]).sum(), 
                                                                  t=len(sdss_tx)))
print('{s} out of {t} XGA Lx52 measurements were successful'.format(s=np.isfinite(sdss_lx[:, 0]).sum(), 
                                                                    t=len(sdss_lx)))
print('{s} out of {t} XGA Lxbol measurements were successful'.format(s=np.isfinite(sdss_lxbol[:, 0]).sum(), 
                                                                    t=len(sdss_lxbol)))

#### $T_{\rm{X}, 500}$, $L^{\rm{0.5-2.0}}_{\rm{X}, 500}$, and $L^{\rm{bol}}_{\rm{X}, 500}$ 

In [ ]:
direct_comparison_plot([loclit_tx, loclit_lxbol], [locuss_txce, locuss_lxbol], 
                       [r"LoCuSS $T_{\rm{X, 500ce}}$ [keV]", r"LoCuSS $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"], 
                       [r"XGA $T_{\rm{X, 500ce}}$ [keV]", r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"],
                       r"LoCuSS High-$L_{\rm{X}}$", (12, 6), ['linear', 'log'], ['linear', 'log'], 
                       savepath='../../outputs/figures/locuss_Txce_Lxbol_comp.pdf')

In [ ]:
fig, ax_arr = plt.subplots(ncols=3, figsize=(18, 6))

# Setting the y-position and font size of the a, b, and c labels that are added below the subplots
sublabel_ypos = -0.15
sublabel_fsize = 14

# Iterating through the array of axes objects, setting up the ticks
for ax_ind, ax in enumerate(ax_arr):
    # Turning on minor ticks and setting it up so they all point inwards - also turn on ticks on the 
    #  top and right axis lines
    ax.minorticks_on()
    ax.tick_params(which='both', top=True, right=True, direction='in')
    # Add the a, b, c, etc labels below the axes
    ax.text(0.5, sublabel_ypos, s='{})'.format(chr(97+ax_ind)), horizontalalignment='center', 
            verticalalignment='center', transform=ax.transAxes, fontsize=sublabel_fsize)

# Setting the leftmost axis to be current
plt.sca(ax_arr[0])
# Using the function we defined earlier to find appropriate axis limits
t_lims = find_lims(xcs3p_tx, sdss_tx, buffer=0.1).value

# Also using the limits to set up a one to one line
# Then plotting the temperature comparison points
plt.plot(t_lims, t_lims, linestyle='dashed', color='red', label="1:1")
plt.errorbar(xcs3p_tx[:, 0].value, sdss_tx[:, 0].value, xerr=xcs3p_tx[:, 1:].T.value, 
             yerr=sdss_tx[:, 1:].T.value, fmt="kx", capsize=2, label="SDSSRM-XCS")
# Setting axis limits
plt.xlim(t_lims)
plt.ylim(t_lims)

# Labels and legend
plt.xlabel(r"XCS3P $T_{\rm{X, 500}}$ [keV]", fontsize=15)
plt.ylabel(r"XGA $T_{\rm{X, 500}}$ [keV]", fontsize=15)
plt.legend(loc='best', fontsize=13)

# Repeating the process with the middle and right plots, for soft band and bolometric luminosities
plt.sca(ax_arr[1])
l_lims = find_lims(xcs3p_lx52, sdss_lx, buffer=0.1).value

plt.plot(l_lims, l_lims, linestyle='dashed', color='red', label="1:1")
plt.errorbar(xcs3p_lx52[:, 0].value, sdss_lx[:, 0].value, xerr=xcs3p_lx52[:, 1:].T.value, 
             yerr=sdss_lx[:, 1:].T.value, fmt="kx", capsize=2, label="SDSSRM-XCS")
plt.xlim(l_lims)
plt.ylim(l_lims)

plt.xscale('log')
plt.yscale('log')

plt.xlabel(r"XCS3P $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", fontsize=15)
plt.ylabel(r"XGA $L^{\rm{0.5-2.0}}_{\rm{X, 500}}$ [erg s$^{-1}$]", fontsize=15)
plt.legend(loc='best', fontsize=13)

plt.sca(ax_arr[2])
l_lims = find_lims(xcs3p_lxbol, sdss_lxbol, buffer=0.1).value

plt.plot(l_lims, l_lims, linestyle='dashed', color='red', label="1:1")
plt.errorbar(xcs3p_lxbol[:, 0].value, sdss_lxbol[:, 0].value, 
             xerr=xcs3p_lxbol[:, 1:].T.value, 
             yerr=sdss_lxbol[:, 1:].T.value, fmt="kx", capsize=2, label="SDSSRM-XCS")
plt.xlim(l_lims)
plt.ylim(l_lims)

plt.xscale('log')
plt.yscale('log')

plt.xlabel(r"XCS3P $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", fontsize=15)
plt.ylabel(r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", fontsize=15)
plt.legend(loc='best', fontsize=13)

plt.tight_layout()

# plt.savefig("non_paper_figures/sdssxcs_Tx_Lxb_Lx52_comp.pdf")
plt.show()